In [ ]:
all_words_freq = {}
    
    for word in title_words_list:
        if word not in all_words:
            all_words_freq[word] = 1
        else:
            all_words_freq[word] += 1
    
    for word in text_words_list:
        if word not in all_words:
            all_words_freq[word] = 1
        else:
            all_words_freq[word] += 1
    
    normal = 0
    for word in all_words_freq:
        frequency = all_words_freq[word]
        weight = 1 + math.log(frequency)
        normal += weight * weight
    
    normalization[doc_id] = normal

In [1]:
#Done
from hazm import *

In [2]:
#Done
import os
import math

In [3]:
#Done
def get_stopwords(directory):
    stopwords = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as f:
            for stopword in f.read().splitlines():
                if stopword not in stopwords:
                    stopwords.append(stopword)
    return stopwords        

In [258]:
#Done
def prepare_text(raw_text):
    prepared_text = []
    normalizer = Normalizer()
    normalized_text = normalizer.normalize(raw_text)
    tokenized_text = word_tokenize(normalized_text)
    stopwords = get_stopwords('stopwords')
    stemmer = Stemmer()
    
    for token in tokenized_text:
        if token not in stopwords:
            stemmed_text = stemmer.stem(token)
            if stemmed_text != "":
                
                prepared_text.append(stemmed_text.strip())

    return prepared_text

In [5]:
#Done
def prepare_text(raw_text):
    prepared_text = []
    stopwords = get_stopwords('stopwords')
    
    normalizer = Normalizer()
    normalized_text = normalizer.normalize(raw_text)
    for stop in stopwords:
        normalized_text = normalized_text.replace(stop, " ")
    tokenized_text = word_tokenize(normalized_text)
    stemmer = Stemmer()
    
    for token in tokenized_text:
        if token not in stopwords:
            stemmed_text = stemmer.stem(token)
            if stemmed_text != "":
                prepared_text.append(stemmed_text)

    return prepared_text

In [6]:
#Done
from xml.dom.minidom import parse

In [8]:
#Done
def get_title(raw_title):
    title = raw_title[0].childNodes[0].data
    return title

In [9]:
#Done
def get_text(raw_text):
    text = raw_text[0].childNodes[0].data
    return text

In [10]:
#Done
def get_id(raw_id):
    doc_id = raw_id[0].childNodes[0].data
    return doc_id

In [11]:
#Done
import json

In [12]:
#Done
positional_index = {}
doc_length = {} #number of words for each documents

def document_length(title_words_list, text_words_list, doc_id):
    all_words = []
    for word in title_words_list:
        if word not in all_words:
            all_words.append(word)
    
    for word in text_words_list:
        if word not in all_words:
            all_words.append(word)
    doc_length[doc_id] = len(all_words)
    

def construct_positional_indexes(docs_path):
    dom = parse(docs_path)
    pages = dom.getElementsByTagName('page')    
    for page in pages:
        title = get_title(page.getElementsByTagName('title'))
        text = get_text(page.getElementsByTagName('text'))
        doc_id = get_id(page.getElementsByTagName('id'))
        
        text_words_list = prepare_text(text)
        title_words_list = prepare_text(title)
        
        document_length(title_words_list, text_words_list, doc_id)
        
        for index, text_word in enumerate(text_words_list):
            if text_word not in positional_index:
                initializer = {doc_id: {'text': []}}
                positional_index[text_word] = initializer
            elif doc_id not in positional_index[text_word]:
                positional_index[text_word][doc_id] = {'text': []}
            elif 'text' not in positional_index[text_word][doc_id]:
                positional_index[text_word][doc_id]['text'] = []
            positional_index[text_word][doc_id]['text'].append(index)

            
        for index, title_word in enumerate(title_words_list):
            if title_word not in positional_index:
                initializer = {doc_id: {'title': []}}
                positional_index[title_word] = initializer
            elif doc_id not in positional_index[title_word]:
                positional_index[title_word][doc_id] = {'title': []}
            elif 'title' not in positional_index[title_word][doc_id]:
                positional_index[title_word][doc_id]['title'] = []
            positional_index[title_word][doc_id]['title'].append(index)
    

construct_positional_indexes('project1_data/data/Persian.xml')

In [13]:
#Done
def get_posting_list(word):
    # Make sure how to get posting list! From reading file or as an argument
    return positional_index[word]

In [14]:
def get_words_with_bigram(bigram):
    # WARNING: not based on slides     
    words = []
    for word in positional_index:
        if bigram in word:
            words.append(word)
    return words
# get_words_with_bigram('لا')

In [15]:
def get_words_with_bigram(bigram):
    # WARNING: based on slides    
    inverted_index = {}
    for word in positional_index:
        word_length = len(word)
        if word_length >= 2:
            for i in range(len(word) - 2):
                bi = word[i: i + 2]
                if bi not in inverted_index:
                    inverted_index[bi] = []
                if word not in inverted_index[bi]:
                    inverted_index[bi].append(word)
    return inverted_index[bigram]
                    
# get_words_with_bigram('لا')

In [17]:
def add_document_to_indexes(docs_path, doc_num):
    address = docs_path + '/' + doc_num + '.xml'
    construct_positional_indexes(address)

# add_document_to_indexes('data/wiki', 20)

In [18]:
def delete_document_from_indexes(docs_path, doc_num):
    address = docs_path + '/' + doc_num + '.xml'
    dom = parse(address)
    pages = dom.getElementsByTagName('page')    
    for page in pages:
        title = get_title(page.getElementsByTagName('title'))
        text = get_text(page.getElementsByTagName('text'))
        doc_id = get_id(page.getElementsByTagName('id'))
        
        text_words_list = prepare_text(text)
        title_words_list = prepare_text(title)
        
        for index, text_word in enumerate(text_words_list):
            if text_word not in positional_index:
                print('کلمه پیدا نشد')
            elif doc_id not in positional_index[text_word]:
                print('چنین کلمه‌ای با این شماره سند موجود نیست')
            else:
                positional_index[text_word].remove(doc_id)
            if len(positional_index[text_word]) == 0:
                del positional_index[text_word]
                
        for index, title_word in enumerate(title_words_list):
            if title_word not in positional_index:
                print('کلمه پیدا نشد')
            elif doc_id not in positional_index[title_word]:
                print('چنین کلمه‌ای با این شماره سند موجود نیست')
            else:
                positional_index[title_word].remove(doc_id)
            if len(positional_index[title_word]) == 0:
                del positional_index[title_word]

# delete_document_from_indexes('data/wiki', 10)

In [19]:
def save_index(destination):
    full_destination = destination + '.json'
    with open(full_destination, 'w') as f:
        json.dump(positional_index, f, ensure_ascii=False)

# save_index('storage/index_backup')

In [20]:
import re

In [270]:
#Done
def positional_intersect(phrase):
    phrases = prepare_text(phrase)
    if len(phrases) == 1:
        return get_posting_list(phrases[0])
    
    phrase_touples = []
    
    for i in range(len(phrases) - 1):
        phrase_touples.append((phrases[i], phrases[i + 1]))
    
    answer = {}
    
    for index, (phrase1, phrase2) in enumerate(phrase_touples):
        doc_list1 = {}
        if index == 0:
            doc_list1 = positional_index[phrase1]
            
        elif not answer:
            return answer
        
        else:
            doc_list1 = answer
            answer = {}

        try:
            doc_list_len1 = len(doc_list1)
            doc_list2 = positional_index[phrase2]
            doc_list_len2 = len(doc_list2)

        except:
            return answer
        i, j = 0, 0
        while i < doc_list_len1 and j < doc_list_len2:
            doc1_keys = list(doc_list1.keys())
            doc2_keys = list(doc_list2.keys())


            doc1_key = doc1_keys[i]
            doc2_key = doc2_keys[j]


            if doc1_key == doc2_key:
                doc_id = doc1_key
                contexts = ['title', 'text']
                for context in contexts:
                    try:
                        context_indices1 = doc_list1[doc1_key][context]
                        context_indices2 = doc_list2[doc2_key][context]
                    except:
                        continue


                    m, n = 0, 0
                    while m < len(context_indices1) and n < len(context_indices2):
                        first_phrase_index = context_indices1[m]
                        second_phrase_index = context_indices2[n]

                        if second_phrase_index - first_phrase_index == 1:
                            if doc_id not in answer:
                                answer[doc_id] = {}

                            if context not in answer[doc_id]:
                                answer[doc_id][context] = []

                            answer[doc_id][context].append(second_phrase_index)

                            m += 1
                            n += 1
                        elif first_phrase_index < second_phrase_index:
                            m += 1
                        else:
                            n += 1
                    i += 1
                    j += 1

            elif doc1_key < doc2_key:
                i += 1
            else:
                j += 1
                
#     print(answer)
    return answer

def get_phrase_occurence(phrase): 
    #count the number of occurence of phrase in docuemnts
    documents = positional_intersect(phrase[:])
    document_occurence = len(list(documents.keys()))
    return document_occurence


In [217]:
# Done
def phrase_normalizer(double_quoted):
    new_double_quotes = []
    for quote in double_quoted:
        quote_list = prepare_text(quote)
        phrase = ""
        for quotes in quote_list:
            phrase += quotes + " "
        new_double_quotes.append(phrase.strip())
    return new_double_quotes

def parsing_query(query):
    all_quotations = re.findall('"([^"]*)"', query)
    query = query.replace('"', "")
    for quoted in all_quotations:
        query = query.replace(quoted, "")
        query = query.replace('  ', " ")
    normalized_phrases = phrase_normalizer(all_quotations[:])
    not_sequential = prepare_text(query)
    return normalized_phrases, not_sequential

In [367]:
# Done 
N = len(doc_length)

def get_quoted_posting_list(phrases):
    first_double_quote = phrases.pop()
    first_double_quote_info = positional_intersect(first_double_quote)
    first_double_quote_docs = list(first_double_quote_info.keys()) #docs containing the first double quote
    answer = first_double_quote_docs
    
    for double_quote in phrases:
        double_quote_info = positional_intersect(double_quote)
        if double_quote_info:            
            other_double_quote_docs = list(double_quote_info.keys())
            new_answer = []
            for ans_doc_id in answer:
                for doc_id_other in other_double_quote_docs:
                    if ans_doc_id == doc_id_other:
                        new_answer.append(ans_doc_id)
            answer = new_answer
        else:
            return []

    return answer


def get_dictionary(query_list):
    dic = {}
    for term in query_list:
        if term not in dic:
            dic[term] = 1
        else:
            dic[term] += 1
    return dic


def consine_score(query_list, method, weight, quoted=False, double_quoted=[], contexts=['title', 'text']):
    if quoted: #new
        valid_docs = get_quoted_posting_list(double_quoted[:]) #docs that have all phrases
        
    else:
        valid_docs = list(doc_length.keys())
        
    scores = {} #score of documents
    query_terms_occurence = get_dictionary(query_list) #terms occurence in query
    terms = list(query_terms_occurence.keys()) #all distinct terms in query
    doc_normalization = 0
    
    print('terms', terms)

    for term in terms:
        if term in positional_index or term in double_quoted:
            if quoted:
                docs_list = positional_intersect(term)
            else:
                docs_list = positional_index[term]

            tf_q = query_terms_occurence[term]
            document_occurence = len(docs_list)
            print('term:', term, 'query occ:', query_terms_occurence[term], 'doc occ:', document_occurence)
            
            idf = math.log(N / document_occurence)
            w_tq = tf_q * idf

            for doc in docs_list:
                if doc in valid_docs:
                    tf = 0
                    for context in contexts:
                        if 'title' in docs_list[doc] and context == 'title':
                            title_length = len(docs_list[doc]['title'])
                            tf += weight * (1 + math.log(title_length))
                        elif 'text' in docs_list[doc] and context == 'text':
                            text_length = len(docs_list[doc]['text'])
                            tf += 1 + math.log(text_length)    
                        w_td = tf
                        doc_normalization += w_td * w_td
                        if doc in scores:
                            scores[doc] += w_tq * w_td
                        else:
                            scores[doc] = w_tq * w_td
    if method == 'ltc-lnc':
        doc_normalization = math.sqrt(doc_normalization)
        for doc in scores:
            scores[doc] /= doc_normalization  
#     scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)} #TODO: not works
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    return sorted_scores[:15]


def search(query, method="ltn-lnn", weight=2, contexts=['title', 'text']):
    print('query', query)
    double_quoted, not_sequential = parsing_query(query)

    all_words = [] #new
    all_words.extend(double_quoted + not_sequential) #new

    if not double_quoted:
        return consine_score(all_words[:], method, weight, contexts=contexts[:])
    
    return consine_score(all_words, method, weight, True, double_quoted[:], contexts=contexts[:])

In [360]:
def detailed_search(title_query, text_query, method="ltn-lnn"):
    relevant_docs = {}
    title_score = search(title_query, method=method, weight=2, contexts=['title'])
    text_score = search(text_query, method=method, weight=1, contexts=['text'])
    for doc_id, score in title_score:
        if doc_id in relevant_docs:
            relevant_docs[doc_id] += score
        else:
            relevant_docs[doc_id] = score
    
    for doc_id, score in text_score:
        if doc_id in relevant_docs:
            relevant_docs[doc_id] += score
        else:
            relevant_docs[doc_id] = score
            
    scores = sorted(relevant_docs.items(), key=lambda item: item[1], reverse=True)
    return scores[:15]

In [410]:
# Done
def get_queries(query_id='all'):
    directory = 'project1_data/data/'
    queries = {}
    relevances = {}
    query_directory = directory + 'queries/'
    relevant_directory = directory + 'relevance/'

    if query_id == 'all':
        for filename in os.listdir(query_directory):
            with open (os.path.join(query_directory, filename), 'r') as f:
                query = f.read()
                queries[filename[:-4]] = query
    
        for filename in os.listdir(relevant_directory):
            with open (os.path.join(relevant_directory, filename), 'r') as f:
                relevance = f.read()
                relevant_docs = relevance.split(', ')
                relevances[filename[:-4]] = relevant_docs
    else:
        query_directory += query_id + '.txt'
        with open (query_directory, 'r') as f:
            query = f.read()
            queries[query_id] = query
        
        relevant_directory += query_id + '.txt'
        with open (relevant_directory, 'r') as f:
            relevance = f.read()
            relevant_docs = relevance.split(', ')
            relevances[query_id] = relevant_docs
    return queries, relevances

queries, relevants = get_queries(query_id='all')

In [417]:
def R_Precision(query_id='all'):
    if query_id == 'all':
        func_queries = queries
        func_relevants = relevants
        keys = list(func_queries.keys())
        query_no = len(keys)

    else:
        func_queries = queries[query_id]
        func_relevants = relevants[query_id]
        keys = [query_id]
        query_no = 1
        
    final_result = 0
        
    for key in keys:
        if '\n' in func_queries[key]:
            title_query, text_query = func_queries[key].split('\n')
            related = detailed_search(title_query, text_query)
        else:
            related = search(func_queries[key])
            
        tp_list = []
        for doc_id, score in related:
            if doc_id in func_relevants[key]:
                tp_list.append(doc_id)
                
        tp = len(tp_list)
           
#         print()
#         print('func_relevants[key]', sorted(func_relevants[key]))
#         print('tp:', tp)
#         print('relevance', len(func_relevants[key]))
#         print()
        
        final_result +=  tp / (len(func_relevants[key]))
        
    return final_result / query_no

print(R_Precision())

query طبیعت دامنه کوه ایرانی
terms ['طبیع', 'دامنه', 'کوه', 'ایران']
term: طبیع query occ: 1 doc occ: 253
term: دامنه query occ: 1 doc occ: 129
term: کوه query occ: 1 doc occ: 251
term: ایران query occ: 1 doc occ: 292
query مطالعه "علوم اجتماعی" در دانشگاه
terms ['علو اجتماع', 'مطالعه', 'در', 'دانشگاه']
term: علو اجتماع query occ: 1 doc occ: 20
term: مطالعه query occ: 1 doc occ: 96
term: در query occ: 1 doc occ: 1240
term: دانشگاه query occ: 1 doc occ: 322
query هیتلر در جنگ جهانی اول
terms ['هیتلر', 'در', 'جنگ', 'جهان', 'اول']
term: هیتلر query occ: 1 doc occ: 46
term: در query occ: 1 doc occ: 1240
term: جنگ query occ: 1 doc occ: 297
term: جهان query occ: 1 doc occ: 279
term: اول query occ: 1 doc occ: 380
query سیاره های بزرگ "منظومه شمسی"
terms ['منظومه شمس', 'سیاره', 'بزرگ']
term: منظومه شمس query occ: 1 doc occ: 15
term: سیاره query occ: 1 doc occ: 25
term: بزرگ query occ: 1 doc occ: 576
query 
terms []
query جنگل های بلوط ایران
terms ['جنگل', 'بلوط', 'ایر']
term: جنگل query occ: 1

In [418]:
def F_measure(query_id='all'):
    if query_id == 'all':
        func_queries = queries
        func_relevants = relevants
        keys = list(func_queries.keys())
        query_no = len(queries)

    else:
        func_queries = queries[query_id]
        func_relevants = relevants[query_id]
        keys = [query_id]
        query_no = 1
    
    result = 0
    
    for key in keys:
        if '\n' in func_queries[key]:
            title_query, text_query = func_queries[key].split('\n')
            related = detailed_search(title_query, text_query)
        else:
            related = search(func_queries[key])
        
        tp_list = []
        for doc_id, score in related:
            if doc_id in func_relevants[key]:
                tp_list.append(doc_id)
                
        tp = len(tp_list)
        p = tp / len(related)
        r = tp / len(func_relevants[key])
        
        if p + r != 0:
            result = result + (2 * p * r)/(p + r)
    
    final_result = result / query_no
    return final_result

print(F_measure())

query طبیعت دامنه کوه ایرانی
terms ['طبیع', 'دامنه', 'کوه', 'ایران']
term: طبیع query occ: 1 doc occ: 253
term: دامنه query occ: 1 doc occ: 129
term: کوه query occ: 1 doc occ: 251
term: ایران query occ: 1 doc occ: 292
query مطالعه "علوم اجتماعی" در دانشگاه
terms ['علو اجتماع', 'مطالعه', 'در', 'دانشگاه']
term: علو اجتماع query occ: 1 doc occ: 20
term: مطالعه query occ: 1 doc occ: 96
term: در query occ: 1 doc occ: 1240
term: دانشگاه query occ: 1 doc occ: 322
query هیتلر در جنگ جهانی اول
terms ['هیتلر', 'در', 'جنگ', 'جهان', 'اول']
term: هیتلر query occ: 1 doc occ: 46
term: در query occ: 1 doc occ: 1240
term: جنگ query occ: 1 doc occ: 297
term: جهان query occ: 1 doc occ: 279
term: اول query occ: 1 doc occ: 380
query سیاره های بزرگ "منظومه شمسی"
terms ['منظومه شمس', 'سیاره', 'بزرگ']
term: منظومه شمس query occ: 1 doc occ: 15
term: سیاره query occ: 1 doc occ: 25
term: بزرگ query occ: 1 doc occ: 576
query 
terms []
query جنگل های بلوط ایران
terms ['جنگل', 'بلوط', 'ایر']
term: جنگل query occ: 1

In [422]:
def MAP(query_id='all'):
    if query_id == 'all':
        func_queries = queries
        func_relevants = relevants
        keys = list(func_queries.keys())
    else:
        func_queries = queries[query_id]
        func_relevants = relevants[query_id]
        keys = [query_id]
    
    ap = 0
    
    for key in keys:
        if '\n' in func_queries[key]:
            title_query, text_query = func_queries[key].split('\n')
            related = detailed_search(title_query, text_query)
        else:
            related = search(func_queries[key])
            
        p_at_k_sum, tp = 0, 0
        for i, (doc_id, score) in enumerate(related):
            if doc_id in func_relevants[key]:
                tp += 1
                p_at_k_sum += tp / (i + 1)
                
        if tp != 0:
            ap += p_at_k_sum / tp
    return ap / len(keys)

print(MAP())

query طبیعت دامنه کوه ایرانی
terms ['طبیع', 'دامنه', 'کوه', 'ایران']
term: طبیع query occ: 1 doc occ: 253
term: دامنه query occ: 1 doc occ: 129
term: کوه query occ: 1 doc occ: 251
term: ایران query occ: 1 doc occ: 292
query مطالعه "علوم اجتماعی" در دانشگاه
terms ['علو اجتماع', 'مطالعه', 'در', 'دانشگاه']
term: علو اجتماع query occ: 1 doc occ: 20
term: مطالعه query occ: 1 doc occ: 96
term: در query occ: 1 doc occ: 1240
term: دانشگاه query occ: 1 doc occ: 322
query هیتلر در جنگ جهانی اول
terms ['هیتلر', 'در', 'جنگ', 'جهان', 'اول']
term: هیتلر query occ: 1 doc occ: 46
term: در query occ: 1 doc occ: 1240
term: جنگ query occ: 1 doc occ: 297
term: جهان query occ: 1 doc occ: 279
term: اول query occ: 1 doc occ: 380
query سیاره های بزرگ "منظومه شمسی"
terms ['منظومه شمس', 'سیاره', 'بزرگ']
term: منظومه شمس query occ: 1 doc occ: 15
term: سیاره query occ: 1 doc occ: 25
term: بزرگ query occ: 1 doc occ: 576
query 
terms []
query جنگل های بلوط ایران
terms ['جنگل', 'بلوط', 'ایر']
term: جنگل query occ: 1

In [434]:
def NDCG(query_id='all'):
    if query_id == 'all':
        func_queries = queries
        func_relevants = relevants
        keys = list(func_queries.keys())
        query_no = len(queries)
    else:
        func_queries = queries[query_id]
        func_relevants = relevants[query_id]
        keys = [query_id]
        query_no= 1
    
    ndcg = 0
    for key in keys:
        if '\n' in func_queries[key]:
            title_query, text_query = func_queries[key].split('\n')
            related = detailed_search(title_query, text_query)
        else:
            related = search(func_queries[key])
        main = 1
        print('func_relevants[key]', func_relevants[key])
        for i, doc_id in enumerate(func_relevants[key]):
            if i == 0:
                continue
            main += 1 / math.log(i + 2)
        
        print('related', related[0])
        for i, doc_id in enumerate(func_relevants[key]):
            if i == 0:
                if doc_id in func_relevants[key]:
                    dcg = 1
                else:
                    dcg = 0
            if doc_id in func_relevants[key]:
                dcg += 1 / math.log(i + 2)
        ndcg += dcg / main
    print('ndcg', ndcg)
    return ndcg/query_no

print(NDCG())

query طبیعت دامنه کوه ایرانی
terms ['طبیع', 'دامنه', 'کوه', 'ایران']
term: طبیع query occ: 1 doc occ: 253
term: دامنه query occ: 1 doc occ: 129
term: کوه query occ: 1 doc occ: 251
term: ایران query occ: 1 doc occ: 292
func_relevants[key] ['6459', '3874', '5381', '6289', '3197', '4081', '3199', '6735', '4094', '5619', '6836', '7143', '3654', '3690', '4339', '5503', '4460', '5620', '6840', '6963', '6497', '3027']
related ('3874', 25.83848887855718)
query مطالعه "علوم اجتماعی" در دانشگاه
terms ['علو اجتماع', 'مطالعه', 'در', 'دانشگاه']
term: علو اجتماع query occ: 1 doc occ: 20
term: مطالعه query occ: 1 doc occ: 96
term: در query occ: 1 doc occ: 1240
term: دانشگاه query occ: 1 doc occ: 322
func_relevants[key] ['6694', '6791', '6973', '5428', '5872', '4388', '6506', '6907', '6634', '5571']
related ('6791', 27.50550315987535)
query هیتلر در جنگ جهانی اول
terms ['هیتلر', 'در', 'جنگ', 'جهان', 'اول']
term: هیتلر query occ: 1 doc occ: 46
term: در query occ: 1 doc occ: 1240
term: جنگ query occ: 1 

In [409]:
a = '2, 4, 5'
a.split(', ')

['2', '4', '5']